In [1]:
import re
import sys; sys.path.append("../")

import pandas as pd

from lib.new_orleans_pd_pprr import clean_pprr, remove_unnamed_cols

In [19]:
pprr = pd.read_excel("../data/New Orleans PD/Complaints Data/New Orleans_PD_PPRR_2019.xls")
pprr = remove_unnamed_cols(pprr)
pprr = clean_pprr(pprr)
pprr.loc[:, "Full Name"] = pprr.loc[:, "First Name"].str.cat(pprr.loc[:, "Last Name"], sep=" ")
pprr.loc[:, "Age"] = 2020 - pprr.loc[:, "Birth Year"]
pprr

,Last Name,Middle Name,First Name,Badge #,Department #,Rank Number #,Rank,Birth Year,Full Name,Age
0,Conway,D,Corliss,00911,2702736,C7134,SENIOR POLICE OFFICER,1963,Corliss Conway,57
1,Kendrick,D,Jamal,01226,2702734,C7110,POLICE OFFICER,1969,Jamal Kendrick,51
2,Marshall,M,Tianay,02101,2702734,C7134,SENIOR POLICE OFFICER,1978,Tianay Marshall,42
3,Weber,L,Tamara,01580,2702750,C7134,SENIOR POLICE OFFICER,1975,Tamara Weber,45
4,Pierce,G,Arlinda,,2702705,U0018,ASSISTANT SUPT OF POLICE,1969,Arlinda Pierce,51
...,...,...,...,...,...,...,...,...,...,...
1172,Miller,M,Patrick,,2702733,C7110,POLICE OFFICER,1991,Patrick Miller,29
1173,Collins,,Cleveland,,2702732,C7110,POLICE OFFICER,1995,Cleveland Collins,25
1174,Broussard,E,Dennis,,2702737,C7110,POLICE OFFICER,1993,Dennis Broussard,27
1175,Condoll,,Tyneisha,,2702737,C7110,POLICE OFFICER,1994,Tyneisha Condoll,26


In [20]:
dup_pprr = pprr[pprr["Full Name"].duplicated(keep=False)]
dup_pprr

,Last Name,Middle Name,First Name,Badge #,Department #,Rank Number #,Rank,Birth Year,Full Name,Age
116,Palumbo,P,Samuel,00462,2702791,C7111,POLICE SERGEANT,1950,Samuel Palumbo,70
123,Williams,A,Derrick,02275,2702736,C7134,SENIOR POLICE OFFICER,1972,Derrick Williams,48
135,Jones,,Lawrence,00209,2702705,C7111,POLICE SERGEANT,1971,Lawrence Jones,49
180,Benjamin,C,Rene,00289,2702782,C7111,POLICE SERGEANT,1965,Rene Benjamin,55
431,Palumbo,,Samuel,00359,2702736,C7112,POLICE LIEUTENANT,1981,Samuel Palumbo,39
476,Jones,,Lawrence,01695,2702731,C7134,SENIOR POLICE OFFICER,1975,Lawrence Jones,45
940,Williams,,Derrick,01465,2702731,C7110,POLICE OFFICER,1983,Derrick Williams,37
1095,Benjamin,C,Rene,602,2702732,C7110,POLICE OFFICER,1990,Rene Benjamin,30


In [22]:
dup_names = dup_pprr["Full Name"].unique()
dup_names

array(['Samuel Palumbo', 'Derrick Williams', 'Lawrence Jones',
       'Rene Benjamin'], dtype=object)

In [13]:
ctn = pd.read_excel("../data/New Orleans PD/Complaints Data/New Orleans_PD_Tracking Number_2020.xls")
ctn = ctn[[col for col in ctn.columns if not col.startswith("Unnamed:")]]
ctn.loc[:, "Full Name"] = ctn.loc[:, "First name"].str.cat(ctn.loc[:, "Last name"], sep=" ")
ctn

,Complaint Tracking Number,Last name,First name,Full Name
0,2020-0001-P,Rigamer,Robert,Robert Rigamer
1,2020-0002-P,NaN,NaN,NaN
2,2020-0003-P,Pfeiffer,Michael,Michael Pfeiffer
3,2020-0004-R,Mcmullen,Daniel,Daniel Mcmullen
4,2020-0005-P,Morris,William,William Morris
...,...,...,...,...
687,2020-0585-P,Godchaux,Kira,Kira Godchaux
688,2020-0590-R,Samuel,Jennifer,Jennifer Samuel
689,2020-0593-P,Arnaud,Lester,Lester Arnaud
690,2020-0597-P,Wiedman,Andrew,Andrew Wiedman


In [31]:
# ctn[ctn["Complaint Tracking Number"] == "2020-0006-P"]
ctn["Complaint Tracking Number"].sort_values().value_counts(dropna=True)

2020-0006-P    21
2020-0207-R     7
2020-0283-P     6
2020-0390-P     6
2020-0313-P     6
               ..
2020-0398-P     1
2020-0213-P     1
2020-0276-P     1
2020-0255-P     1
2020-0051-R     1
Name: Complaint Tracking Number, Length: 446, dtype: int64

In [15]:
cprr = pd.read_csv("../data/New Orleans PD/Complaints Data/New Orleans_PD_CPRR_2020.csv")
cprr

,Incident Type,Complaint Tracking Number,Date Complaint Occurred,Date Complaint Received by NOPD (PIB),Date Complaint Investigation Complete,Complaint classification,Investigation status,Disposition,Bureau of Complainant,Division of Complainant,...,Rule Violation,Paragraph Violation,Unique Officer Allegation ID,Officer Race Ethnicity,Officer Gender,Officer Age,Officer years of service,Complainant Gender,Complainant Ethnicity,Complainant Age
0,Public Initiated,2020-0001-P,10/19/2014,1/3/2020,NaN,DI-1,Forwarded,Pending,FOB - Field Operations Bureau,2nd District,...,RULE 2: MORAL CONDUCT,PARAGRAPH 01 - ADHERENCE TO LAW,55485.0,White,Male,45,0.0,Male,NaN,NaN
1,Public Initiated,2020-0002-P,10/27/2019,1/3/2020,3/6/2020,DI-1,Completed,Not Sustained,NaN,NaN,...,RULE 3: PROF CONDUCT,PARAGRAPH 01 - Professionalism,57204.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN
2,Public Initiated,2020-0003-P,1/1/2020,1/3/2020,5/4/2020,DI-1,Completed,Unfounded,NaN,NaN,...,RULE 3: PROF CONDUCT,PARAGRAPH 01 - Professionalism,56365.0,White,Male,66,0.0,NaN,Black,NaN
3,Rank Initiated,2020-0004-R,1/3/2020,1/15/2020,NaN,DI-1,Forwarded,Pending,ISB - Investigations and Support Bureau,Criminal Investigations Division,...,RULE 4: PERF OF DUTY,PARAGRAPH 02 - INSTRUCTIONS FROM AUTHORITATIVE...,55421.0,White,Male,52,0.0,NaN,NaN,NaN
4,Public Initiated,2020-0005-P,1/4/2020,1/5/2020,8/31/2020,DI-1,Completed,Sustained,FOB - Field Operations Bureau,4th District,...,RULE 4: PERF OF DUTY,PARAGRAPH 04 - NEGLECT OF DUTY,56821.0,Black,Male,33,0.0,Female,Black,30.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
756,Rank Initiated,2020-0625-R,12/19/2020,12/20/2020,NaN,DI-1,Active,Pending,FOB - Field Operations Bureau,5th District,...,RULE 4: PERF OF DUTY,PARAGRAPH 04 - NEGLECT OF DUTY,57954.0,Black,Female,33,0.0,NaN,NaN,NaN
757,Rank Initiated,2020-0625-R,12/19/2020,12/20/2020,NaN,DI-1,Active,Pending,FOB - Field Operations Bureau,5th District,...,RULE 4: PERF OF DUTY,PARAGRAPH 04 - NEGLECT OF DUTY,57956.0,Black,Male,22,0.0,NaN,NaN,NaN
758,Rank Initiated,2020-0628-R,12/11/2019,12/22/2020,NaN,DI-1,Initial,Pending,FOB - Field Operations Bureau,3rd District,...,RULE 4: PERF OF DUTY,PARAGRAPH 04 - NEGLECT OF DUTY,57986.0,White,Male,38,0.0,Male,White,76.0
759,Rank Initiated,2020-0628-R,12/11/2019,12/22/2020,NaN,DI-1,Initial,Pending,FOB - Field Operations Bureau,3rd District,...,RULE 4: PERF OF DUTY,PARAGRAPH 04 - NEGLECT OF DUTY,57987.0,White,Female,22,0.0,Male,White,76.0


In [16]:
cprr.columns

Index(['Incident Type', 'Complaint Tracking Number', 'Date Complaint Occurred',
       'Date Complaint Received by NOPD (PIB)',
       'Date Complaint Investigation Complete', 'Complaint classification',
       'Investigation status', 'Disposition', 'Bureau of Complainant',
       'Division of Complainant', 'Unit of Complainant',
       'Unit Additional Details of Complainant',
       'Working Status of Complainant', 'Shift of Complainant',
       'Rule Violation', 'Paragraph Violation', 'Unique Officer Allegation ID',
       'Officer Race Ethnicity', 'Officer Gender', 'Officer Age',
       'Officer years of service', 'Complainant Gender',
       'Complainant Ethnicity', 'Complainant Age'],
      dtype='object')

In [35]:
# cprr[cprr["Complaint Tracking Number"] == "2020-0006-P"]
sr1 = ctn["Complaint Tracking Number"].sort_values().value_counts(dropna=True)
sr2 = cprr["Complaint Tracking Number"].sort_values().value_counts(dropna=True)
for ind, val in sr1.items():
    if ind not in sr2:
        continue
    if val != sr2[ind]:
        print("%s: %d != %d" % (ind, val, sr2[ind]))

2020-0575-P: 1 != 2
2020-0414-R: 1 != 2


In [27]:
cprr[cprr["Unique Officer Allegation ID"].duplicated()]

,Incident Type,Complaint Tracking Number,Date Complaint Occurred,Date Complaint Received by NOPD (PIB),Date Complaint Investigation Complete,Complaint classification,Investigation status,Disposition,Bureau of Complainant,Division of Complainant,...,Rule Violation,Paragraph Violation,Unique Officer Allegation ID,Officer Race Ethnicity,Officer Gender,Officer Age,Officer years of service,Complainant Gender,Complainant Ethnicity,Complainant Age
508,Rank Initiated,2020-0409-R,NaN,8/17/2020,NaN,DI-1,Active,Pending,MSB - Management Services Bureau,NaN,...,RULE 3: PROF CONDUCT,PARAGRAPH 04 - Accepting,NaN,Giving Anything of Value,White,Male,NaN,0,NaN,NaN
509,Rank Initiated,2020-0409-R,NaN,8/17/2020,NaN,DI-1,Active,Pending,MSB - Management Services Bureau,NaN,...,RULE 3: PROF CONDUCT,PARAGRAPH 04 - Accepting,NaN,Giving Anything of Value,Black,Female,NaN,0,NaN,NaN
678,Rank Initiated,2020-0560-R,NaN,11/9/2020,NaN,DI-1,Initial,Pending,ISB - Investigative Services Bureau,Reserve Division,...,Unknown Shift Hours,RULE 3: PROF CONDUCT,NaN,PARAGRAPH 01 - Professionalism,NaN,NaN,NaN,0,NaN,NaN
679,Rank Initiated,2020-0560-R,NaN,11/9/2020,NaN,DI-1,Initial,Pending,ISB - Investigative Services Bureau,Reserve Division,...,Unknown Shift Hours,RULE 3: PROF CONDUCT,NaN,PARAGRAPH 01 - Professionalism,Black,Female,NaN,0,NaN,NaN
